## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
import scipy.io as sio
from matplotlib.ticker import ScalarFormatter, MaxNLocator
#%matplotlib notebook
import seaborn as sns

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from brainpipe.feature import power, amplitude, sigfilt
import scipy.stats as stats

## Bootstrap Statistics

In [2]:
def boot_matrix(z, B):
    """Bootstrap sample
    Returns all bootstrap samples in a matrix, random trials (some repeated)
    Do not change the size of the matrix always len(z) » len(z)*B"""
    n = len(z)  # sample size
    idz = np.random.randint(0, n, size=(B, n))  # indices to pick for all boostrap samples
    return z[idz]

def bootstrap_mean(x, B=10000, alpha=0.05):
    """Bootstrap standard error and (1-alpha)*100% c.i. for the population mean
    Returns bootstrapped standard error and different types of confidence intervals"""
    # 2D array, loop for each time points
    x_boot_wins, x_sd_boot, x_ci_boot = np.array([]),np.array([]),np.array([])
    for i in range(x.shape[1]):
        # Deterministic things
        x_win = x[:,i]
        n = len(x_win)  # sample size
        orig = x_win.mean()  # sample mean
        se_mean = x_win.std()/np.sqrt(n) # standard error of the mean
        qt = stats.t.ppf(q=1 - alpha/2, df=n - 1) # Student quantile
        # Generate boostrap distribution of sample mean
        xboot = boot_matrix(x_win, B=B)
        xboot = xboot[:,:,np.newaxis]
        sampling_distribution = xboot.mean(axis=1)
        # Standard error and sample quantiles
        se_mean_boot = sampling_distribution.std()
        quantile_boot = np.percentile(sampling_distribution, q=(100*alpha/2, 100*(1-alpha/2)))
        # Concatenate all values
        x_boot_wins = np.concatenate((x_boot_wins,xboot), axis=2) if np.size(x_boot_wins) else xboot
        x_sd_boot = np.vstack((x_sd_boot,se_mean_boot)) if np.size(x_sd_boot) else se_mean_boot
        x_ci_boot = np.vstack((x_ci_boot,quantile_boot)) if np.size(x_ci_boot) else quantile_boot
    print(x_boot_wins.shape, x_sd_boot.shape, x_ci_boot.shape)
    return x_boot_wins, x_sd_boot, x_ci_boot

## User variables

In [3]:
# where to find data
st = study('Olfacto')
score = 'Epi' #'Rec'
if score == 'Epi':
    path_elec = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_Good_Bad_EpiScore/')
    path_pow = path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_700_100_EpiScore/')
    save_pow = path.join(path_pow, 'Bootstrap_10000_Power_balanced_rest/')
# ANALYSIS PARAMETERS
minsucc = 5 #nb of continuous samples to be significant

-> Olfacto loaded


## Power Decoding - Good Bad Odors Encoding

In [4]:
test = False

if test == True:
    n_elec = {'PIRJ' :2}
    subjects = ['PIRJ']
    nfreq = 1
else :
    subjects = ['MICP','VACJ','SEMC','PIRJ','LEFC','CHAF'] 
    nfreq = 5
    n_elec = {
    'CHAF' : 81,
    'VACJ' : 91, 
    'SEMC' : 81,
    'PIRJ' : 62,
    'LEFC' : 160,
    'MICP' : 79,
        }

for su in subjects:
    #load power files (nfreq, nelec, nwin, ntrial)
    bad_data = np.load(path.join(path_pow, su+'_concat_odor_bad_bipo_new_rest_power.npz'))['xpow']
    good_data = np.load(path.join(path_pow, su+'_concat_odor_good_bipo_new_rest_power.npz'))['xpow']
    print (su, 'bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)

# ==========================  BALANCED CONDITIONS - Bootstrap  =====================================
    if bad_data.shape[3] > good_data.shape[3]:
        bad_stat = bad_data[:,:,:,np.random.randint(bad_data.shape[3], size=good_data.shape[3])]
        good_stat = good_data
    elif bad_data.shape[3] < good_data.shape[3]:
        bad_stat = bad_data
        good_stat = good_data[:,:,:,np.random.randint(good_data.shape[3], size=bad_data.shape[3])]
    else:
        bad_stat, good_stat = bad_data, good_data
    ntrials = bad_stat.shape[3]
    print ('balanced data : ', bad_stat.shape, good_stat.shape)
    
# =========================== COMPUTE for 1 elec 1 freq ========================================    
    for elec_num in range(n_elec[su]):
        for freq in range(nfreq):
            # load power files for 1 elec // 1 freq // Bad-Good conditions
            bad_data_elec = bad_stat[freq,elec_num].swapaxes(0,1)
            good_data_elec = good_stat[freq,elec_num].swapaxes(0,1)
            print ('data elec ', bad_data_elec.shape, good_data_elec.shape)
            ntrials = str(bad_data_elec.shape[0])+'/'+ str(good_data_elec.shape[0])
            nwin = good_data_elec.shape[1]
            elec = np.load(path.join(path_elec, su+'_concat_odor_bad_bipo_new.npz'))['channel'][elec_num]
            elec_label = np.load(path.join(path_elec, su+'_concat_odor_bad_bipo_new.npz'))['label'][elec_num]
            freq_name = np.load(path.join(path_pow, su+'_concat_odor_bad_bipo_new_rest_power.npz'))['fname'][freq]
            print ('nwin',nwin,'elec ', elec, 'elec_label ', elec_label)
            
#================================ BOOTSTRAP AND STATISTICS ====================================        
            # Bootstrap and Welsh t-test
            bad_boot, sd_bad_boot, ci_bad_boot = bootstrap_mean(bad_data_elec, B=10000)
            good_boot, sd_good_boot, ci_good_boot = bootstrap_mean(good_data_elec, B=10000)            
            print('bootstrap mat : ', bad_boot.shape, good_boot.shape)
            # Data to average
            bad_mean = np.mean(bad_data_elec, axis=0)
            good_mean = np.mean(good_data_elec, axis=0)
            # Create a threshold vector
            th_vals = []
            for t in range(nwin):
                if ci_bad_boot[t,0] >= good_mean[t]:
                    th_vals.append(0.04)
                elif ci_bad_boot[t,1] <= good_mean[t]:
                    th_vals.append(0.04)
                else:
                    th_vals.append(1)
            print(len(th_vals), th_vals)
            
#========================== PREPARE PLOTS AND SAVE STATS =========================================
            # plot and figure parameters
            xfmt = ScalarFormatter(useMathText=True)
            xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(9,6))
            rmaxis(plt.gca(), ['right', 'top'])
            title = 'Bootstrap Power for '+su+' '+score+' Good/Bad '+str(elec)+' '+str(elec_label)+' ('+str(elec_num)+') ntrials:'+str(ntrials)
            fig.suptitle(title, fontsize=12)
            # Time vector to plot power
            step = 3500/ good_data_elec.shape[1]
            times_plot = np.arange(-500, 3000, step)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            # Plot the Bootstrap STER with confidence intervals
            plt.plot(times_plot, bad_mean, '-', color='b', label='bad')
            plt.plot(times_plot, good_mean, '-', color='m', label='good')
            plt.legend(loc=0, handletextpad=0.1, frameon=False) 
            plt.xlabel('Time (ms)')
            plt.ylabel('Single trial evoked response (mV)')
            plt.fill_between(times_plot, ci_bad_boot[:,0],ci_bad_boot[:,1], alpha=0.2, color='b')
            plt.fill_between(times_plot, ci_good_boot[:,0],ci_good_boot[:,1], alpha=0.2, color='m')
            addPval(plt.gca(), th_vals, p=0.05, x=times_plot, y=2, color='orange', lw=2, minsucc=minsucc)
            addLines(plt.gca(), vLines=[0], vWidth=[2], vColor=['#000000'], hLines=[0], 
                     hColor=['#000000'], hWidth=[2])
            # Significance criteria to reach      
            pvals = np.ravel(th_vals)
            underp = np.where(pvals < 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= minsucc ]

            #Save plots and stats
            if len(signif) >=1:
                name_bad_boot = (save_pow+str(freq)+'_'+freq_name+'/Significant/data/'+su +'_bad_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')                                    
                name_ci_bad_boot = (save_pow+str(freq)+'_'+freq_name+'/Significant/data/'+su +'_ci_bad_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                name_good_boot = (save_pow+str(freq)+'_'+freq_name+'/Significant/data/'+su +'_good_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                name_ci_good_boot = (save_pow+str(freq)+'_'+freq_name+'/Significant/data/'+su +'_ci_good_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                plot_name = (save_pow+str(freq)+'_'+freq_name+'/Significant/fig/'+su +'_STER_' + score +'_'+str(elec_label)+'_('+str(elec)+').png')
            else:
                name_bad_boot = (save_pow+str(freq)+'_'+freq_name+'/Not_Significant/data/'+su +'_bad_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                name_ci_bad_boot = (save_pow+str(freq)+'_'+freq_name+'/Not_Significant/data/'+su +'_ci_bad_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                name_good_boot = (save_pow+str(freq)+'_'+freq_name+'/Not_Significant/data/'+su +'_good_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                name_ci_good_boot = (save_pow+str(freq)+'_'+freq_name+'/Not_Significant/data/'+su +'_ci_good_boot_' + score +'_'+str(elec_label)+'_('+str(elec)+').npy')
                plot_name = (save_pow+str(freq)+'_'+freq_name+'/Not_Significant/fig/'+su +'_STER_' + score +'_'+str(elec_label)+'_('+str(elec)+').png')

            np.save(name_bad_boot, bad_boot)
            np.save(name_ci_bad_boot, ci_bad_boot)
            np.save(name_good_boot, good_boot)
            np.save(name_ci_good_boot, ci_good_boot)
            plt.savefig(plot_name, dpi=300, bbox_inches='tight')
            plt.clf()
            plt.close()       

MICP bad shape:  (5, 79, 13, 46) good shape:  (5, 79, 13, 37)
balanced data :  (5, 79, 13, 37) (5, 79, 13, 37)
data elec  (37, 13) (37, 13)
nwin 13 elec  a'2-a'1 elec_label  Amg&Amg-pPirT
(10000, 37, 13) (13, 1) (13, 2)
(10000, 37, 13) (13, 1) (13, 2)
bootstrap mat :  (10000, 37, 13) (10000, 37, 13)
13 [0.04, 1, 1, 1, 1, 0.04, 0.04, 1, 1, 1, 1, 1, 1]
data elec  (37, 13) (37, 13)
nwin 13 elec  a'2-a'1 elec_label  Amg&Amg-pPirT
(10000, 37, 13) (13, 1) (13, 2)
(10000, 37, 13) (13, 1) (13, 2)
bootstrap mat :  (10000, 37, 13) (10000, 37, 13)
13 [1, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 1, 1, 1, 1, 0.04]
data elec  (37, 13) (37, 13)
nwin 13 elec  a'2-a'1 elec_label  Amg&Amg-pPirT
(10000, 37, 13) (13, 1) (13, 2)
(10000, 37, 13) (13, 1) (13, 2)
bootstrap mat :  (10000, 37, 13) (10000, 37, 13)
13 [0.04, 0.04, 1, 1, 0.04, 0.04, 0.04, 0.04, 1, 1, 1, 1, 0.04]
data elec  (37, 13) (37, 13)
nwin 13 elec  a'2-a'1 elec_label  Amg&Amg-pPirT
(10000, 37, 13) (13, 1) (13, 2)
(10000, 37, 13) (13, 1) (13